In [ ]:
import os
from pathlib import Path
import cv2 as cv 
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, BatchNormalization, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential

In [ ]:
# make paths
current_path = os.getcwd()
train_path = Path( os.path.join(( os.path.dirname(os.path.dirname(current_path)) ), "Research", "data", "train") )
test_path = Path( os.path.join(( os.path.dirname(os.path.dirname(current_path)) ), "Research", "data", "test") )
valid_path = Path( os.path.join(( os.path.dirname(os.path.dirname(current_path)) ), "Research", "data", "valid") )

In [ ]:
# check the number of classes (i.e., number of species) in train, test and validation datasets
number_of_classes = len(os.listdir(train_path))
print(number_of_classes)
number_of_classes = len(os.listdir(test_path))
print(number_of_classes)
number_of_classes = len(os.listdir(valid_path))
print(number_of_classes)

In [ ]:
# prepare for visualizing the images
image_files = [f for bird_species in train_path.iterdir() for f in bird_species.glob('*.jpg')]
image_df = pd.DataFrame({
    'Filepath': image_files,
    'Label': [f.parent.name for f in image_files]  # using the parent folder name as label
})

In [ ]:
# read in the CSV file
birds_df = pd.read_csv(os.path.join(os.path.dirname(train_path), "birds.csv"))
labels_to_scientific_name = dict(zip(birds_df['labels'], birds_df['scientific name']))

In [ ]:
# visualize 16 random images
random_index = np.random.randint(0, len(image_df), 16)
fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(12, 12),
                        subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(image_df.Filepath.iloc[random_index[i]]))
    
    # fetch the scientific name using the label and the created dictionary
    label = image_df.Label.iloc[random_index[i]]
    scientific_name = labels_to_scientific_name.get(label, "Unknown")    
    ax.set_title(f"{label}\n({scientific_name})")  # display both common and scientific names
    
plt.tight_layout()
plt.show()

In [ ]:
DataGenerator = ImageDataGenerator(rescale=1./255)

In [ ]:
train_data = DataGenerator.flow_from_directory(train_path, target_size=(224,224),batch_size=32,class_mode='categorical')
test_data = DataGenerator.flow_from_directory(test_path, target_size=(224,224),batch_size=32,class_mode='categorical')
valid_data = DataGenerator.flow_from_directory(valid_path, target_size=(224,224),batch_size=32,class_mode='categorical')

In [ ]:
# base_model = tf.keras.applications.InceptionV3(weights='imagenet', include_top= False,)
# base_model = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(224,224,3))

### Config the base model

In [ ]:
base_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet')
base_model.trainable = False
inputs = tf.keras.layers.Input(shape=(224,224,3), name="input-layer")
X = base_model(inputs)
X = tf.keras.layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(X)
outputs = tf.keras.layers.Dense(524, activation="softmax", name="output-layer")(X)
model_0 = tf.keras.Model(inputs, outputs)

In [ ]:
# base_model.trainable = False

In [ ]:
# inputs = tf.keras.layers.Input(shape =(224,224,3), name = "input-layer")

In [ ]:
# X = base_model(inputs)
# print(f"Shape after passing inputs through base model: {X.shape}")

In [ ]:
# X = tf.keras.layers.GlobalAveragePooling2D(name = "global_average_pooling_layer")(X)
# print(f"Shape after GlobalAveragePooling2D: {X.shape}")

In [ ]:
# outputs = tf.keras.layers.Dense(524, activation = "softmax", name = "output-layer")(X)

In [ ]:
# model_0 = tf.keras.Model(inputs, outputs)

In [ ]:
model_0.compile(loss = "categorical_crossentropy",
                optimizer = tf.keras.optimizers.Adam(learning_rate = 0.01),
                metrics = ["accuracy"])

In [ ]:
model_0.summary()

In [ ]:
history = model_0.fit(train_data,
                      epochs=10,
                      steps_per_epoch = len(train_data),
                      validation_data = valid_data,
                      validation_steps = int(0.25*len(valid_data)),)

In [ ]:
model_0.evaluate(test_data)

### Fine tune the model

In [ ]:
base_model.trainable = True

# Un-freeze last 10 layers
for layer in base_model.layers[:-10]:
    layer.trainable = False

In [ ]:
# Recompile
model_0.compile(loss = "categorical_crossentropy",
                optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001),
                metrics = ["accuracy"] )

In [ ]:
print(len(model_0.trainable_variables))

In [ ]:
initial_epochs = 10
fine_tune_epochs = initial_epochs + 1

# Refit the model
history_2 = model_0.fit(train_data,
                        epochs = fine_tune_epochs,
                        validation_data = valid_data,
                        validation_steps = int(0.25*len(valid_data)),
                        initial_epoch =  history.epoch[-1],) # start the epoch where it left before

In [ ]:
model_0.evaluate(test_data)

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
import sys

sys.path.append("../")  # add parent directory to the system path
from utils_data import *
from utils_model import *
from utils_YOLOv8 import *
from ultralytics import  YOLO

In [ ]:
path_to_image = "../input/data/real_imgs/001.jpg"

detect_model = YOLO("yolov8n.pt")
results = detect_model(path_to_image, show=False, save=False)
bboxes = results[0].boxes.xyxy.cpu().numpy().reshape(-1, 4)
print(bboxes)

if bboxes.size ==0:
    print("true")

In [ ]:
path_to_chosen_model = ("../models/YOLOv8/test0/train/weights/last.pt")
# model = YOLO("yolov8n-cls.pt")
model = YOLO(path_to_chosen_model)
# Export the model
# model.export(format="onnx")

In [ ]:
path_to_image = ("../input/data/test/ALBATROSS/1.jpg")
path_to_chosen_model = ("../models/YOLOv8/test0/train/weights/last.onnx")
model = YOLO(path_to_chosen_model)
results = model(path_to_image)